In [2]:
import lmppl
import pandas as pd, numpy as np
from transformers import AutoTokenizer

checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
# scorer = lmppl.LM(checkpoint)
# scorer = lmppl.LM("Qwen/Qwen2-7B-Instruct")
# scorer = lmppl.LM("google/gemma-2-9b-it")

In [3]:
class FormattingFunction:
    def __init__(self, model_checkpoint):
        self.instruction_all = {
            "meta-llama/Meta-Llama-3-8B-Instruct": """
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        당신의 역할은 한국어로 답변하는 **한국어 AI 어시트턴트**입니다. 주어진 질문에 대해 한국어로 답변해주세요.<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        아래 질문을 한국어로 정확하게 답변해주세요. **질문**: {}<|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>\n\n{}<|eot_id|>
        <|end_of_text|>""",
            "google/gemma-2-9b-it": """
        <bos><start_of_turn>model
        당신의 역할은 한국어로 답변하는 **한국어 AI 어시트턴트**입니다. 주어진 질문에 대해 한국어로 답변해주세요.<end_of_turn>
        <start_of_turn>user
        아래 질문을 한국어로 정확하게 답변해주세요. **질문**: {}<end_of_turn>
        <start_of_turn>model
        {}<end_of_turn>
        <eos>""",
            "Qwen/Qwen2-7B-Instruct": """
        <|im_start|>system
        당신의 역할은 한국어로 답변하는 **한국어 AI 어시트턴트**입니다. 주어진 질문에 대해 한국어로 답변해주세요.\n<|im_end|>
        <|im_start|>user
        아래 질문을 한국어로 정확하게 답변해주세요. **질문**: {}<|im_end|>
        <|im_start|>system
        {}<|im_end|>
        <|endoftext|>""",
        }

        self.instruction = self.instruction_all[model_checkpoint]

    def __call__(self, examples):

        final_texts = []
        for i in range(len(examples["input"])):
            final_text = self.instruction.format(
                examples["input"][i], examples["output"][i]
            )
            final_texts.append(final_text)

        return final_texts


class DataBase:
    def __init__(self):
        train_url = "/node_storage2/data_llm_kr/data_it_train_240724.csv"
        eval_url = "/node_storage2/data_llm_kr/data_it_eval_240724.csv"

        eval_translate_url = "/home/hyohyeongjang/bigdata/ModelTrainer/llm-kr/data/oig-smallchip2-dedu-slice_reviewed_week1-7_instruction_valid.csv"

        self.train_df = pd.read_csv(train_url)
        self.eval_df = pd.read_csv(eval_url)
        self.eval_translate = pd.read_csv(eval_translate_url)


d = DataBase()
f = FormattingFunction(checkpoint)

/tmp/ipykernel_36040/1090207392.py:51: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  self.train_df = pd.read_csv(train_url)


In [4]:
def count_tokens(sentences):
    target = sentences.apply(lambda x: f"{x['input']} {x['output']}", axis=1).tolist()

    checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    out_llama = tokenizer(target)
    out_llama = pd.Series([len(i) for i in out_llama["input_ids"]], name="llama_token")

    checkpoint = "Qwen/Qwen2-7B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    out_qwen = tokenizer(target)
    out_qwen = pd.Series([len(i) for i in out_qwen["input_ids"]], name="qwen_token")

    checkpoint = "google/gemma-2-9b-it"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    out_gemma = tokenizer(target)
    out_gemma = pd.Series([len(i) for i in out_gemma["input_ids"]], name="gemma_token")

    x = pd.concat([sentences, out_llama, out_qwen, out_gemma], axis=1)
    x.groupby("task")[["llama_token", "qwen_token", "gemma_token"]].agg(
        ["mean", "max", "min"]
    ).round(1)

    return x


def get_perplexity(sentences, checkpoint_type, sample=True, sample_num=100):

    # sentences = d.train_df
    if sample:
        samples = (
            sentences[["input", "output", "task"]]
            .groupby("task")
            .sample(sample_num, random_state=21)
            .reset_index(drop=True)
        )
    else:
        samples = sentences

    # samples = sentences
    targets = f(samples)

    ppl = np.array(scorer.get_perplexity(targets, batch=8)).round(2)
    x = pd.concat([samples, pd.Series(ppl, name=checkpoint_type)], axis=1)
    x.groupby("task")[checkpoint_type].mean()

    return x

In [ ]:
if __name__ == "__main__":

    # count_tokens(d.train_df).to_csv("/home/hyohyeongjang/bigdata/ModelTrainer/llm-kr/data/token_stat.csv")

    # tasks = d.train_df.task.drop_duplicates().tolist()

    # partial_task = len(tasks) // 5

    # ppl_lst = []
    # for i in range(5):
    #     if i == 4:
    #         this_task = tasks[i * partial_task :]
    #     else:
    #         this_task = tasks[i * partial_task : (i + 1) * partial_task]

    #     sentences = d.train_df[d.train_df.task.map(lambda x: x in this_task)]
    #     sentences = d.train_df.loc[
    #         d.train_df.task.map(
    #             lambda x: x
    #             in [
    #                 "doosan_253717",
    #                 "gen_argument",
    #                 "gen_news_classification",
    #                 "gen_sentiment_classification",
    #                 "gen_sts_classification",
    #                 "med_translation"
    #                 "mec_definition"
    #             ]
    #         )
    #     ].reset_index(drop=True)

    #     ppl_out = get_perplexity(sentences, sample_num = 10, checkpoint_type="llama_ppl")
    #     ppl_lst.append(ppl_out)

  0%|                                 | 0/63 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
 19%|████▌                   | 12/63 [01:45<07:30,  8.82s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 11.69 GiB. GPU 

In [ ]:
[
    "doosan_253717",
    "gen_argument",
    "gen_news_classification",
    "gen_sentiment_classification",
    "gen_sts_classification",
    "med_translation" "mec_definition",
]

In [5]:
d.train_df.groupby("task").count()

,instruction,input,output,option
task,,,,
doosan_253717,0,228345,228345,0
fin_definition,0,669,669,0
gen_argument,0,11871,11871,0
gen_assistant,17385,18000,18000,0
gen_closedQA_classification,0,900,900,0
gen_closedQA_generation_1,0,900,900,0
gen_closedQA_generation_2,13390,13390,13390,13390
gen_closedQA_generation_3,18000,18000,18000,0
gen_generation,0,11574,11574,0


In [6]:
x = pd.read_csv("/home/hyohyeongjang/bigdata/ModelTrainer/llm-kr/data/token_stat.csv")

In [7]:
x

,Unnamed: 0,llama_token,llama_token.1,llama_token.2,qwen_token,qwen_token.1,qwen_token.2,gemma_token,gemma_token.1,gemma_token.2
0,NaN,mean,max,min,mean,max,min,mean,max,min
1,task,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,doosan_253717,60.7,6350,7,66.2,6722,6,65.3,6357,7
3,fin_definition,330.4,714,67,359.2,754,72,369.3,784,71
4,gen_argument,221.2,24921,10,241.2,26011,10,248.2,27280,11
5,gen_assistant,102.1,9621,26,111.1,11216,25,113.4,11022,28
6,gen_closedQA_classification,162.5,317,56,180.9,353,59,180.0,360,64
7,gen_closedQA_generation_1,342.5,7357,50,387.3,8457,58,380.2,8055,60
8,gen_closedQA_generation_2,412.1,1211,117,456.4,1335,116,466.0,1351,134
9,gen_closedQA_generation_3,516.2,3267,130,571.2,3601,143,562.8,2478,147
